## Parse cert chain
See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.load_pem_x509_certificates.

Handy.

In [1]:
from cryptography import x509

In [2]:
with open("google-com-chain.pem", "rb") as pem_file:
    certs = x509.load_pem_x509_certificates(pem_file.read())

In [3]:
assert len(certs) == 3 # Expect 3 certificates in the chain.

In [4]:
for idx, cert in enumerate(certs):
    print(f"cert {idx}, Subject: {cert.subject} Issuer: {cert.issuer}")

cert 0, Subject: <Name(CN=*.google.com)> Issuer: <Name(C=US,O=Google Trust Services,CN=WE2)>
cert 1, Subject: <Name(C=US,O=Google Trust Services,CN=WE2)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>
cert 2, Subject: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>


In [5]:
from cryptography.hazmat.primitives.asymmetric import ec

pyca API seems pretty user-friendly. See code example at https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.tbs_certificate_bytes.

Also see https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ec/#elliptic-curve-signature-algorithms.

Note ECDSA(hash algorithm) for `verify` call.

In [ ]:
# Verify certificate is signed by the issuer given its public key.
def verify_issuer_sig(cert_to_check, issuer_public_key):
    issuer_public_key.verify(
        cert_to_check.signature,
        cert_to_check.tbs_certificate_bytes,
        ec.ECDSA(cert_to_check.signature_hash_algorithm),
    )
    # Expect verify() to throw an exception if the signature is invalid. If we get here, the signature is valid.
    print(f"Verified cert with subject {cert_to_check.subject} is signed by issuer {cert_to_check.issuer}.")

## Verify leaf cert

In [11]:
verify_issuer_sig(certs[0], certs[1].public_key())

Verified cert with subject <Name(CN=*.google.com)> is signed by issuer <Name(C=US,O=Google Trust Services,CN=WE2)>.


## Verify sig on sub-CA cert

In [12]:
verify_issuer_sig(certs[1], certs[2].public_key())

Verified cert with subject <Name(C=US,O=Google Trust Services,CN=WE2)> is signed by issuer <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>.


## Tinker with root cert

In [13]:
is_self_signed = certs[2].subject == certs[2].issuer
is_self_signed

True